<a href="https://colab.research.google.com/github/tagasy/kaggle/blob/master/lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
X_train = train.drop(['id','target'], axis=1)
y_train = train['target']

In [0]:
#評価用関数
def get_evaluate(y_test, predict):

    fpr, tpr, thr_arr = metrics.roc_curve(y_test, predict)

    auc = metrics.auc(fpr, tpr)
    precision = metrics.precision_score(y_test, predict)
    recall = metrics.recall_score(y_test, predict)      

    return auc, precision, recall

In [0]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

params = {
    'objective': 'binary',
    'seed': 71,
    'learning_rate': 0.05,
    'verbose': 1,
    'num_leaves': 20,
    'metrics': 'binary_logloss'

}
auc_list = []
precision_list = []
recall_list = []

for train_index, test_index in enumerate(folds.split(X_train, y_train)):
#ValueError: Found input variables with inconsistent numbers of samples: [300, 250]って出る
  X_train, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
  y_train, y_test = y_train.iloc[train_index], y_train.iloc[test_index]

  lgb_train = lgb.Dataset(X_train, y_train)
  lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

  model = lgb.train(params, 
                    lgb_train,
                    num_boost_round=300,
                    valid_sets=[lgb_train,lgb_test],
                    early_stopping_rounds=10)

  predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
  predict = [0 if i < 0.5 else 1 for i in predict_proba]

  auc, precision, recall = get_evaluate(y_test, predict)
  print('AUC:{}, precision:{}, recall:{}'.format(auc, precision, recall))

  auc_list.append(auc)
  precision_list.append(precision)
  recall_list.append(recall)

print('Kfold平均 AUC:{}, precision:{}, recall:{}'.format(np.mean(auc_list), 
                                                         np.mean(precision_list), 
                                                         np.mean(recall_list)))